# Setting up the environment and starting midi generation

We connect the disk to save the output files to it. You will need to enter a token from your account.

Download the MuseGAN project from GitHub, set up a virtual environment, put all the necessary versions of the libraries into it. Install CUDA9 in the Colab environment.

In [ ]:
!git clone https://github.com/chessnocow/musegan

!pip install pipenv
!pipenv install

!pipenv run pip install setuptools==39.1.0
!wget https://github.com/tensorflow/tensorflow/archive/v1.10.1.tar.gz
!pipenv run pip install v1.10.1.tar.gz
!pipenv run pip install -r musegan/requirements.txt

!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1

!musegan/scripts/download_models.sh

!pip install pypianoroll==1.0.0

In [ ]:
!pipenv run pip install -r ts.txt

In [ ]:
!chmod 755 -R musegan/scripts

In [ ]:
!musegan/scripts/download_data.sh

In [ ]:
!pipenv run musegan/scripts/process_data.sh

In [ ]:
!pipenv run musegan/scripts/setup_exp.sh "musegan/exp/my_experiment1/"

In [ ]:
!pipenv run musegan/scripts/run_train.sh "musegan/exp/my_experiment1/" "0"

In [ ]:
!musegan/scripts/collect_data.sh "midi/" "data/train.npy"

Run the inference script in the pipenv virtual environment. The npz output files are saved in the project folder.

In [ ]:
!pipenv run musegan/scripts/run_inference.sh "musegan/exp/default/" "0"

Run the interpolation script. The number of launches is written in the script file, by default 10. The output npz files are saved in the project folder.

In [ ]:
!pipenv run musegan/scripts/run_interpolation.sh "musegan/exp/default/" "0"

Roll back the pypianoroll version in order to convert the npz files.

In [ ]:
!pip install pypianoroll==0.5.2

  Created wheel for pypianoroll: filename=pypianoroll-0.5.2-cp36-none-any.whl size=23489 sha256=43f34dec214b299a07ea4c6c3e1165e3e12f236be631726a483a836c4247f79d
  Stored in directory: /root/.cache/pip/wheels/bf/dd/f9/7084a29342f2fe5fb3c48f35f333f0b106f418999ae63a9890
Successfully built pypianoroll
  Found existing installation: pypianoroll 1.0.0
    Uninstalling pypianoroll-1.0.0:
      Successfully uninstalled pypianoroll-1.0.0


Defining functions for converting npz files to midi.

In [ ]:
import glob
import music21 as m21
import pypianoroll as ppr
import os
from pypianoroll import Multitrack

def open_midi(midi_path, remove_drums=False):
  """
  Opens midi file from midi_path as a music21.Stream object.
  """
  mf = m21.midi.MidiFile()
  mf.open(midi_path)
  mf.read()
  mf.close()
  if (remove_drums):
    for i in range(len(mf.tracks)):
      mf.tracks[i].events = [ev for ev in mf.tracks[i].events if ev.channel != 10]          
  return m21.midi.translate.midiFileToStream(mf)

def save_midi_from_npz(npz_filename, midi_filename, n_bins=4):
  """
  Converts npz file into midi. Also cuts every n_bins + 1 measure of midi. 
  Every 5th bin is silence in MuseGAN.
  """
  midi_npz = Multitrack(npz_filename)
  midi_npz.write(midi_filename)
  midifile = open_midi(midi_filename)
  try:
    midi_measures = midifile.measures(1, 9999999)
  except:
    print("Can't convert this!")
    os.remove(midi_filename)
    return 0
  midi_cut = m21.stream.Score()
  
  indexes = 
  for part in midi_measures:
    partnew = m21.stream.Part()
    for i, el in enumerate(part):
      if (i + 1) % (n_bins + 1) != 0:
        partnew.append(el)
    midi_cut.append(partnew)
  midi_cut.write('midi', midi_filename)

def cut_midi_in_measures(input_file, n_bins=4, n_parts=10):
  """
  Cuts midi files by n_bins partitions.
  """
  midifile = open_midi(input_file)
  print("File opened")
  for i in range(n_parts):
    try:  
      midi_temp = m21.stream.Score()
      midi_temp.append(midifile.measures(i * n_bins, i * n_bins + n_bins))
      print(input_file[:-4] + "_" + str(i) + ".mid")
      midi_temp.write('midi', input_file[:-4] + "_" + str(i) + ".mid")
    except:
      break


In [ ]:
!pip install mido

Converting all files in output folders. Not all files are converted, some of them cannot be converted with pauses removed.

There are 4 types of files in total:

* inference hard_thresholding
* inference bernoulli_sampling
* interpolation hard_thresholding
* interpolation bernoulli_sampling
As a result, we get midi files in a folder on the musegan / midi disk.

In [ ]:
import glob
for file in glob.glob("musegan/exp/default/results/inference/pianorolls/fake_x_hard_thresholding/*.npz"):
  save_midi_from_npz(file, 'musegan/midi/' + file.split('/')[-1] + '_inf_hard.mid')
  
for file in glob.glob("musegan/exp/default/results/inference/pianorolls/fake_x_bernoulli_sampling/*.npz"):
  save_midi_from_npz(file, 'musegan/midi/' + file.split('/')[-1] + '_inf_bern.mid')

for file in glob.glob("musegan/exp/default/results/interpolation/pianorolls/fake_x_bernoulli_sampling/*.npz"):
  save_midi_from_npz(file, 'musegan/midi/' + file.split('/')[-1] + '_inter_bern.mid')

for file in glob.glob("musegan/exp/default/results/interpolation/pianorolls/fake_x_hard_thresholding/*.npz"):
  save_midi_from_npz(file, 'musegan/midi/' +  file.split('/')[-1] + '_inter_hard.mid')

# Cutting, gluing midi files

Functions for cutting midi, merging midi and getting midi of a certain duration from a set of files

In [ ]:
from mido import MidiFile, MidiTrack, tick2second
import glob

def cut_midi(input_file, output_file, start_time, end_time) :
    """
    Cuts midi input_file from start_time to end_time and saves cutted part into 
    output_file.
    """
    input_midi = MidiFile(input_file)
    output_midi = MidiFile()
    tempo = 600000
    if input_midi.type == 2 :
        print("Can't cut the file")
        
    # Copying the time metrics between both files
    output_midi.ticks_per_beat = input_midi.ticks_per_beat

    for original_track in input_midi.tracks :
        for msg in original_track :
            if msg.type == 'set_tempo' : 
                print(msg.tempo)
                tempo = msg.tempo
                break    
    
    for original_track in input_midi.tracks :
        new_track = MidiTrack()
        total_time = 0
        for msg in original_track :
            if msg.type in ['note_on', 'note_off'] :
                total_time  += tick2second(msg.time, input_midi.ticks_per_beat, tempo)
                if total_time < start_time or total_time > end_time : continue
            new_track.append(msg)
        output_midi.tracks.append(new_track)
    
    output_midi.save(output_file)


def join_midi(input_files, output_file):
    """
    Takes list of input_files and joins it into one output_file.
    input_files - list of paths to files
    output_file - path to the output file
    """
    output_midi = MidiFile()
    tempo = 600000
    new_tracks = []
    for idx, input_file in enumerate(input_files):
      input_midi = MidiFile(input_file)
      if input_midi.type == 2 :
        continue  
      # Copying the time metrics between both files
      if idx == 0:
        output_midi.ticks_per_beat = input_midi.ticks_per_beat
        for original_track in input_midi.tracks :
          for msg in original_track :
            if msg.type == 'set_tempo' : 
              print(msg.tempo)
              tempo = msg.tempo
              break    
      
      for track_n, original_track in enumerate(input_midi.tracks) :
        if idx == 0:
          new_tracks.append(MidiTrack())
        for msg in original_track :
          new_tracks[track_n].append(msg)
    for track in new_tracks:
      output_midi.tracks.append(track)
    
    output_midi.save(output_file)


def join_midi_to_length(input_folder, output_file, length):
  """
  Takes midi files from input folder and joins these to length seconds.
  You can use a wildcard in input folder, for example:
  "/content/1*.mid" for files in content folder which names start 
  from 1 and end with .mid 
  """
  output_midi = MidiFile()
  tempo = 600000
  new_tracks = []
  tracks_length = []
  for idx, file in enumerate(glob.glob(input_folder)):
    input_midi = MidiFile(file)
    if input_midi.type == 2 :
        continue  
      # Copying the time metrics between both files
    if idx == 0:
      output_midi.ticks_per_beat = input_midi.ticks_per_beat
      for original_track in input_midi.tracks :
        for msg in original_track :
          if msg.type == 'set_tempo' : 
            tempo = msg.tempo
            break
    for track_n, original_track in enumerate(input_midi.tracks):
      if idx == 0:
        new_tracks.append(MidiTrack())
        tracks_length.append(0)
      for msg in original_track :
        new_tracks[track_n].append(msg)
        if msg.type in ['note_on', 'note_off'] :
          tracks_length[track_n]  += tick2second(msg.time, input_midi.ticks_per_beat, tempo)
          if tracks_length[track_n] > length : break
  for track in new_tracks:
    output_midi.tracks.append(track)
    
  output_midi.save(output_file)
           
        


In [ ]:
input_folder = "musegan/midi/*inter_bern.mid"
output_file = "cut.mid"
join_midi_to_length(input_folder, output_file, 240)

In [ ]:
input_file = "musegan/midi/fake_x_bernoulli_sampling_0.npz_inf_bern.mid"
output_file = "cut_2.mid"
cut_midi(input_file, output_file, start_time=10, end_time=50)

In [ ]:
input_files = ["musegan/midi/fake_x_bernoulli_sampling_0.npz_inf_bern.mid",
               "musegan/midi/fake_x_bernoulli_sampling_0.npz_inf_bern_0.mid",
               "musegan/midi/fake_x_hard_thresholding_0.npz_inf_hard.mid",
               "musegan/midi/fake_x_bernoulli_sampling_5.npz_inf_bern.mid"
              ]
output_file = "join.mid"
join_midi(input_files, output_file)

# Replacing notes with chords

In [ ]:
from mido import MidiFile, MidiTrack, tick2second

def notes_to_chords(input_file, output_file, tracks=[1, 2, 3, 4, 5], chords="major") :
    """
    Changes single notes in midi to chords.
    input_file  - path to the input midi file
    output_file - path to the output file to save
    tracks - list of tracks numers to modify
    chords - "major" or "minor" for corresponding triads
    """
    input_midi = MidiFile(input_file)
    output_midi = MidiFile()

    # Copying the time metrics between both files
    output_midi.ticks_per_beat = input_midi.ticks_per_beat

    for track_n, original_track in enumerate(input_midi.tracks):
      if track_n in  tracks:
        new_track = MidiTrack()
        cur = []
        for idx, msg in enumerate(original_track):
          if msg.type == "note_on":
            cur.append(msg)
          elif msg.type == "note_off":
            cur.append(msg)
            if len(cur) != 2:
              for el in cur:
                new_track.append(el)
              cur=[]
            else:
              note = cur[0].note
              if chords == "major":
                new_track.append(cur[0].copy())
                new_track.append(cur[0].copy(note=note+4, time=0))
                new_track.append(cur[0].copy(note=note+7, time=0))
                new_track.append(cur[1])
                new_track.append(cur[1].copy(note=note+4, time=0))
                new_track.append(cur[1].copy(note=note+7, time=0))
              elif chords == "minor":
                new_track.append(cur[0].copy())
                new_track.append(cur[0].copy(note=note+3, time=0))
                new_track.append(cur[0].copy(note=note+7, time=0))
                new_track.append(cur[1])
                new_track.append(cur[1].copy(note=note+3, time=0))
                new_track.append(cur[1].copy(note=note+7, time=0))
              cur=[]
          else:
            new_track.append(msg)
        output_midi.tracks.append(new_track)

    output_midi.save(output_file)

In [ ]:
input_file = "31.mid"
output_file = "out.mid"
notes_to_chords(input_file, output_file, [1], "minor")

# Changing insturments

We define the function of changing instruments, it also uses the function of opening midi from this cell.

In [ ]:
from music21 import converter,instrument, midi


def change_instruments(input_file, output_file, new_instruments):
  """
  Opens midi file from input_file, change its instruments to new_instruments. 
  The new instruments should be a list with 5 music21.instrument.Instrument objects. 
  For example instrument.Piano(), instrument.Bass() and so on.
  Function saves file in output_file path.
  """
  s = open_midi(input_file)
  for i, part in enumerate(s):
    for el in part.recurse():
      if isinstance(el, instrument.Instrument):
        el.activeSite.replace(el, new_instruments[i])

  s.write('midi', output_file)

An example of using the tool change function

In [ ]:
input_file = "midi2wav/midi/fake_x_hard_thresholding_0.npz_inter_hard.mid"
output_file = "midi2wav/midi/__fake_x_hard_thresholding_0.npz_inter_hard.mid"
new_instruments = [instrument.SnareDrum(), instrument.Piano(), instrument.ElectricBass(), instrument.ElectricGuitar(), instrument.Guitar()]

change_instruments(input_file, output_file, new_instruments)

# Convert midi to wav using various sound fonts

Install fluidsynth

In [ ]:
!apt-get install fluidsynth

We define the conversion function.

In [ ]:
from midi2audio import FluidSynth


def convert_midi_to_wav(input_file, output_file, soundfont=""):
  """
  Converts midi file from input_file to wav output_file with particulary soundfont.
  You have to install fluidsynth utility first. The soundfont waits for 
  a whole path to sf2 file.
  """
  fs = FluidSynth(soundfont)
  fs.midi_to_audio(input_file, output_file)

An example of using the function to convert midi to wav. Google Drive must be connected.

In [ ]:
sf = "midi2wav/soundfonts/Anologue Heaven.SF2"
input_file = "midi2wav/midi/fake_x_hard_thresholding_9.npz_inter_hard.mid"
output_file = "midi2wav/wav/test1.wav"
convert_midi_to_wav(input_file, output_file, sf)